In [2]:
import torch_geometric
from torch_geometric.datasets import Planetoid

## Load datset

In [3]:
dataset = Planetoid(root="tutorial", name="Cora")

Processing...
Done!


In [4]:
print("Number of graphs:\t", len(dataset))
print("Number of classes:\t", dataset.num_classes)
print("Number of node features:\t", dataset.num_node_features)
print("Number of edge features:\t", dataset.num_edge_features)

Number of graphs:	 1
Number of classes:	 7
Number of node features:	 1433
Number of edge features:	 0


In [5]:
print(dataset.data)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


## Build model

In [16]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv

In [7]:
data = dataset[0]

In [17]:
class GraphNet(torch.nn.Module):
    def __init__(self):
        super(GraphNet, self).__init__()
        self.conv = SAGEConv(dataset.num_features,
                             dataset.num_classes,
                             aggr="max")
        
    def forward(self):
        x = self.conv(data.x, data.edge_index)
        return F.log_softmax(x, dim=1)

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GraphNet().to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(),
                             lr=0.01,
                             weight_decay=5e-4)

## Train model

In [19]:
def train():
    model.train()
    optimizer.zero_grad()
    out = model()[data.train_mask]
    y = data.y[data.train_mask]
    loss = F.nll_loss(out, y)
    grad = loss.backward()
    optimizer.step()
                  

In [25]:
def test():
    model.eval()
    logits = model()
    accs = []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

In [27]:
best_val_acc = 0
test_acc = 0

for epoch in range(1, 100):
    train()
    _, val_acc, tmp_test_acc = test()
    
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    
    log = "Epoch: {:03d}, Val: {:.4f}, Test: {:.4f}"
    
    if epoch%10 == 0:
        print(log.format(epoch, best_val_acc, test_acc))

Epoch: 010, Val: 0.7200, Test: 0.7170
Epoch: 020, Val: 0.7200, Test: 0.7170
Epoch: 030, Val: 0.7200, Test: 0.7170
Epoch: 040, Val: 0.7200, Test: 0.7170
Epoch: 050, Val: 0.7220, Test: 0.7160
Epoch: 060, Val: 0.7220, Test: 0.7160
Epoch: 070, Val: 0.7220, Test: 0.7160
Epoch: 080, Val: 0.7220, Test: 0.7160
Epoch: 090, Val: 0.7240, Test: 0.7260
